# Part 4: Dynamic Network Analysis & Enhanced Feature Engineering for Forecasting

This notebook continues from Part 2, utilizing the `trade_data_features.csv` file. The primary goal is to enrich our dataset with features derived from the dynamic, year-over-year evolution of the global trade network. These features are crucial for building more accurate and insightful predictive models.

The objectives are:
1.  **Iteratively Construct Yearly Trade Networks**: For each year from 1988 to 2024, build a directed, weighted graph representing trade flows.
2.  **Calculate Dynamic Centrality Measures**: Compute various centrality metrics (PageRank, HITS, Harmonic, Betweenness) for each country within each yearly network, generating a total of 2,967 country-year centrality records.
3.  **Engineer Rolling Statistics for Centralities**: To capture trends and volatility, calculate 3-year rolling means and standard deviations of these centrality scores.
4.  **Perform Dynamic Community Detection**: Employ the Louvain algorithm on yearly networks to identify evolving trade communities, resulting in 2,967 country-year community assignments.
5.  **Engineer Features from Dynamic Communities**: Derive features such as `community_stability` to quantify how often a country changes its community membership.
6.  **Integrate Dynamic Features**: Merge all 34 newly created dynamic features (for both importer and exporter perspectives) back into the main dataset, expanding it to 825 columns.
7.  **Lag Features for Forecasting**: Lag all 34 dynamic features by one year to prevent data leakage in predictive models.
8.  **Handle Missing Values**: Address `NaN` values introduced by rolling, lagging, or incomplete data using median imputation.
9.  **Save Enriched Data**: Output the final, feature-engineered DataFrame with 859 columns, ready for predictive modeling in Part 5.

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import community as community_louvain # for Louvain algorithm
from sklearn.preprocessing import MinMaxScaler # Though not used directly here, good to have for potential normalization if needed later
import os
import warnings

# Configuration
INPUT_FILE = 'trade_data_features.csv'  # Output from Part 2
FINAL_OUTPUT_FILE_PART4 = 'trade_data_dynamic_features.csv' # Final output for Part 4
ROLLING_WINDOW_DYN = 3 # Rolling window for dynamic feature statistics (e.g., 3 years)

# Determine years for analysis (e.g., all available or a specific recent range)
START_YEAR_DYNAMIC_ANALYSIS = None # Auto-detect from data
END_YEAR_DYNAMIC_ANALYSIS = None   # Auto-detect from data

warnings.filterwarnings("ignore", category=RuntimeWarning) # For potential warnings from NetworkX/community on certain graph structures
pd.set_option('display.max_columns', 100) # Show more columns in outputs

## 1. Load Data and Initial Preparation

We begin by loading the dataset produced in Part 2. This file (`trade_data_features.csv`) already contains initial time-series features (lags, rolling stats on 'amount') and `tsfresh` features, with missing values imputed via MICE. The initial shape is (17170, 791). We then automatically detect the full range of available years (1988 to 2024) to be used for the dynamic analysis.

In [2]:
df = pd.read_csv(INPUT_FILE)
print(f"Loaded data from {INPUT_FILE}. Initial Shape: {df.shape}")
print("Initial DataFrame Info:")
df.info(verbose=True, show_counts=True)

# Ensure 'year' is integer and 'trade_pair_id' exists (created in Part 2)
df['year'] = df['year'].astype(int)
if 'trade_pair_id' not in df.columns:
    print("Warning: 'trade_pair_id' not found. Creating it now from 'importer' and 'exporter'.")
    df['trade_pair_id'] = df['importer'] + '_' + df['exporter']

available_years = sorted(df['year'].unique())
if not START_YEAR_DYNAMIC_ANALYSIS:
    START_YEAR_DYNAMIC_ANALYSIS = min(available_years)
if not END_YEAR_DYNAMIC_ANALYSIS:
    END_YEAR_DYNAMIC_ANALYSIS = max(available_years)

analysis_years = [yr for yr in available_years if START_YEAR_DYNAMIC_ANALYSIS <= yr <= END_YEAR_DYNAMIC_ANALYSIS]
print(f"\nYears selected for dynamic analysis: {min(analysis_years)} to {max(analysis_years)} ({len(analysis_years)} years).")
print("DataFrame head after initial preparation:")
print(df.head())

Loaded data from trade_data_features.csv. Initial Shape: (17170, 791)
Initial DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17170 entries, 0 to 17169
Data columns (total 791 columns):
 #    Column                                                                  Non-Null Count  Dtype  
---   ------                                                                  --------------  -----  
 0    importer                                                                17170 non-null  object 
 1    exporter                                                                17170 non-null  object 
 2    amount                                                                  17170 non-null  float64
 3    year                                                                    17170 non-null  int64  
 4    importer_latitude                                                       17170 non-null  float64
 5    importer_longitude                                                      1717

## 2. Calculate Dynamic Centrality Measures (Iterative per Year)

We iterate through each year from 1988 to 2024. For each year, a directed trade graph is constructed, and a suite of centrality measures is calculated for every country (node). This process captures the evolving structural importance of each nation in the global trade network. The key measures are:
    *   **PageRank**: Measures a country's importance as a trade destination.
    *   **HITS (Hubs and Authorities)**: Identifies influential exporters (hubs) and importers (authorities).
    *   **Harmonic Centrality**: Quantifies a country's overall network accessibility.
    *   **Betweenness Centrality**: Measures a country's role as a trade intermediary or bridge.
The results are stored in a `dynamic_centralities_df` DataFrame containing **2,967** country-year records.

In [3]:
yearly_centralities_list = []
print(f"\nStarting calculation of dynamic centralities for years {min(analysis_years)}-{max(analysis_years)}...")

for current_year in analysis_years:
    print(f"  Processing centralities for year: {current_year}")
    df_year = df[df['year'] == current_year]
    
    if df_year.empty:
        print(f"    No trade data for year {current_year}. Skipping centrality calculation.")
        continue
        
    G_year = nx.DiGraph()
    for _, row in df_year.iterrows():
        # Ensure amount is positive for sensible weighting
        if row['amount'] > 0:
            G_year.add_edge(row['exporter'], row['importer'], weight=row['amount'])
    
    if G_year.number_of_nodes() == 0:
        print(f"    Graph for year {current_year} has no nodes (e.g. all amounts were <=0). Skipping centrality calculation.")
        continue

    # Calculate centralities with error handling for robustness
    try:
        pagerank = nx.pagerank(G_year, weight='weight', alpha=0.85)
    except Exception as e:
        print(f"    Warning: PageRank calculation failed for {current_year} ({e}). Defaulting to 0.")
        pagerank = {n: 0.0 for n in G_year.nodes()}
        
    try:
        # HITS can be sensitive to graph structure; max_iter and tol might need adjustment for convergence
        hits_h, hits_a = nx.hits(G_year, max_iter=500, tol=1e-03) 
    except Exception as e: 
        print(f"    Warning: HITS calculation failed for {current_year} ({e}). Defaulting to 0.")
        hits_h = {n: 0.0 for n in G_year.nodes()}
        hits_a = {n: 0.0 for n in G_year.nodes()}
        
    try:
        # Harmonic centrality is generally robust
        harmonic_centrality = nx.harmonic_centrality(G_year)
    except Exception as e:
        print(f"    Warning: Harmonic centrality calculation failed for {current_year} ({e}). Defaulting to 0.")
        harmonic_centrality = {n: 0.0 for n in G_year.nodes()}
    
    try:
      # Betweenness can be computationally intensive on large graphs
      betweenness_centrality = nx.betweenness_centrality(G_year, weight='weight', normalized=True)
    except Exception as e:
      print(f"    Warning: Weighted Betweenness calculation failed for {current_year} ({e}). Trying unweighted.")
      try:
          betweenness_centrality = nx.betweenness_centrality(G_year, normalized=True)
      except Exception as e2:
          print(f"    Warning: Unweighted Betweenness calculation also failed for {current_year} ({e2}). Defaulting to 0.")
          betweenness_centrality = {n: 0.0 for n in G_year.nodes()}

    year_centralities_for_df = []
    all_nodes_in_year = set(df_year['importer']).union(set(df_year['exporter'])) # Ensure all countries present in data for the year are covered
    for node in all_nodes_in_year:
        year_centralities_for_df.append({
            'country': node,
            'year': current_year,
            'pagerank_dyn': pagerank.get(node, 0), # Use .get for nodes that might be in df_year but not form edges
            'hub_score_dyn': hits_h.get(node, 0),
            'authority_score_dyn': hits_a.get(node, 0),
            'harmonic_centrality_dyn': harmonic_centrality.get(node, 0),
            'betweenness_centrality_dyn': betweenness_centrality.get(node, 0)
        })
    yearly_centralities_list.extend(year_centralities_for_df)

dynamic_centralities_df = pd.DataFrame(yearly_centralities_list)
if not dynamic_centralities_df.empty:
    print("\nDynamic centralities calculation complete.")
    print(f"Shape of dynamic_centralities_df: {dynamic_centralities_df.shape}")
    print("Sample of calculated dynamic centralities:")
    print(dynamic_centralities_df.head())
    print("Basic stats for calculated dynamic centralities:")
    print(dynamic_centralities_df.describe())
else:
    print("\nWarning: No dynamic centralities were calculated. `dynamic_centralities_df` is empty.")
    # Create an empty DataFrame with expected columns if it's empty to prevent downstream errors
    if dynamic_centralities_df.empty:
        dynamic_centralities_df = pd.DataFrame(columns=['country', 'year', 'pagerank_dyn', 'hub_score_dyn', 
                                                         'authority_score_dyn', 'harmonic_centrality_dyn', 
                                                         'betweenness_centrality_dyn'])



Starting calculation of dynamic centralities for years 1988-2024...
  Processing centralities for year: 1988
  Processing centralities for year: 1989
  Processing centralities for year: 1990
  Processing centralities for year: 1991
  Processing centralities for year: 1992
  Processing centralities for year: 1993
  Processing centralities for year: 1994
  Processing centralities for year: 1995
  Processing centralities for year: 1996
  Processing centralities for year: 1997
  Processing centralities for year: 1998
  Processing centralities for year: 1999
  Processing centralities for year: 2000
  Processing centralities for year: 2001
  Processing centralities for year: 2002
  Processing centralities for year: 2003
  Processing centralities for year: 2004
  Processing centralities for year: 2005
  Processing centralities for year: 2006
  Processing centralities for year: 2007
  Processing centralities for year: 2008
  Processing centralities for year: 2009
  Processing centralities for

## 3. Engineering Temporal Dynamics: Rolling Statistics for Centralities

Raw yearly centrality scores can be noisy. To capture more stable trends and the volatility of a country's network position, we compute rolling statistics (mean and standard deviation) for each centrality measure. These are calculated per country over a 3-year period (`ROLLING_WINDOW_DYN`).

-   **Rolling Mean**: Smooths out short-term fluctuations, highlighting longer-term trends in centrality.
-   **Rolling Standard Deviation**: Measures the volatility or consistency of a country's centrality score over the window.

`min_periods=1` is used to ensure values are generated even at the start of a time series, with `NaN`s from single-period standard deviations filled with 0.

In [4]:
if not dynamic_centralities_df.empty and 'pagerank_dyn' in dynamic_centralities_df.columns:
    print(f"\nCalculating rolling statistics for centralities with window size {ROLLING_WINDOW_DYN}...")
    dynamic_centralities_df.sort_values(['country', 'year'], inplace=True)
    
    centrality_cols_to_roll = ['pagerank_dyn', 'hub_score_dyn', 'authority_score_dyn', 'harmonic_centrality_dyn', 'betweenness_centrality_dyn']
    
    for col in centrality_cols_to_roll:
        if col in dynamic_centralities_df.columns:
            # Rolling Mean
            dynamic_centralities_df[f'{col}_roll_mean'] = dynamic_centralities_df.groupby('country')[col].transform(
                lambda x: x.rolling(window=ROLLING_WINDOW_DYN, min_periods=1).mean()
            )
            # Rolling Standard Deviation
            dynamic_centralities_df[f'{col}_roll_std'] = dynamic_centralities_df.groupby('country')[col].transform(
                lambda x: x.rolling(window=ROLLING_WINDOW_DYN, min_periods=1).std()
            )
            # Fill NaNs for std (when min_periods=1, std is NaN) with 0, as there's no deviation with a single point.
            dynamic_centralities_df[f'{col}_roll_std'].fillna(0, inplace=True)
        else:
            print(f"    Column {col} not found in dynamic_centralities_df, skipping rolling stats for it.")
            
    print("\nRolling statistics for centralities added.")
    print("Sample of dynamic centralities with rolling stats:")
    # Display a sample that might show the effect of rolling window
    sample_country_for_rolling_view = dynamic_centralities_df['country'].unique()[0] if len(dynamic_centralities_df['country'].unique()) > 0 else None
    if sample_country_for_rolling_view:
        print(dynamic_centralities_df[dynamic_centralities_df['country'] == sample_country_for_rolling_view].head(ROLLING_WINDOW_DYN + 2))
    else:
        print(dynamic_centralities_df.head())
else:
    print("\nSkipping rolling statistics calculation as `dynamic_centralities_df` is empty or lacks base centrality columns.")


Calculating rolling statistics for centralities with window size 3...


C:\Users\Askeladd\AppData\Local\Temp\ipykernel_30800\3664432143.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dynamic_centralities_df[f'{col}_roll_std'].fillna(0, inplace=True)
C:\Users\Askeladd\AppData\Local\Temp\ipykernel_30800\3664432143.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always b


Rolling statistics for centralities added.
Sample of dynamic centralities with rolling stats:
         country  year  pagerank_dyn  hub_score_dyn  authority_score_dyn  \
467  Afghanistan  2000           0.0            0.0                  0.0   
546  Afghanistan  2001           0.0            0.0                  0.0   
624  Afghanistan  2002           0.0            0.0                  0.0   
775  Afghanistan  2003           0.0            0.0                  0.0   
845  Afghanistan  2004           0.0            0.0                  0.0   

     harmonic_centrality_dyn  betweenness_centrality_dyn  \
467                      0.0                         0.0   
546                      0.0                         0.0   
624                      0.0                         0.0   
775                      0.0                         0.0   
845                      0.0                         0.0   

     pagerank_dyn_roll_mean  pagerank_dyn_roll_std  hub_score_dyn_roll_mean  \
467     

C:\Users\Askeladd\AppData\Local\Temp\ipykernel_30800\3664432143.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dynamic_centralities_df[f'{col}_roll_std'].fillna(0, inplace=True)


## 4. Dynamic Community Detection (Louvain Algorithm)

To understand how trade relationships cluster countries into blocs and how these clusters evolve, we perform dynamic community detection. For each year, an undirected, weighted graph is constructed, and the Louvain algorithm (`community_louvain.best_partition`) is applied to find the optimal community structure that maximizes modularity. This process is repeated for all years, resulting in a `dynamic_communities_df` DataFrame with **2,967** country-year community assignments.

In [5]:
yearly_communities_list = []
print(f"\nStarting dynamic community detection for years {min(analysis_years)}-{max(analysis_years)}...")

for current_year in analysis_years:
    print(f"  Processing communities for year: {current_year}")
    df_year = df[df['year'] == current_year]
    
    if df_year.empty:
        print(f"    No trade data for year {current_year}. Skipping community detection.")
        continue
        
    G_year_undir = nx.Graph() # Louvain works on undirected graphs
    for _, row in df_year.iterrows():
        exporter, importer, amount = row['exporter'], row['importer'], row['amount']
        if amount > 0:
            if G_year_undir.has_edge(exporter, importer):
                G_year_undir[exporter][importer]['weight'] += amount
            else:
                G_year_undir.add_edge(exporter, importer, weight=amount)
            
    if G_year_undir.number_of_nodes() == 0:
        print(f"    Graph for year {current_year} has no nodes. Skipping community detection.")
        continue
    
    partition = {}
    if G_year_undir.number_of_edges() > 0:
        try:
            partition = community_louvain.best_partition(G_year_undir, weight='weight', random_state=42)
        except Exception as e:
            print(f"    Warning: Louvain community detection failed for {current_year} ({e}). Assigning nodes to default community -1.")
            partition = {node: -1 for node in G_year_undir.nodes()}
    else: # No edges, assign each node to its own community or a default 'isolated' community
        print(f"    Graph for year {current_year} has no edges. Assigning nodes to default community -1.")
        partition = {node: -1 for node in G_year_undir.nodes()} # Or node: i for i, node in enumerate(G_year_undir.nodes())
        
    all_nodes_in_year = set(df_year['importer']).union(set(df_year['exporter']))
    for node in all_nodes_in_year:
        yearly_communities_list.append({
            'country': node,
            'year': current_year,
            'community_id_dyn': partition.get(node, -2) # -2 if node was in df_year but not in graph (e.g. only zero-amount trades)
        })

dynamic_communities_df = pd.DataFrame(yearly_communities_list)
if not dynamic_communities_df.empty:
    print("\nDynamic community detection complete.")
    print(f"Shape of dynamic_communities_df: {dynamic_communities_df.shape}")
    print("Sample of detected dynamic communities:")
    print(dynamic_communities_df.head())
    print("Value counts for community IDs (sample from first few years):")
    print(dynamic_communities_df[dynamic_communities_df['year'].isin(analysis_years[:3])]['community_id_dyn'].value_counts().head(10))
else:
    print("\nWarning: No dynamic communities were detected. `dynamic_communities_df` is empty.")
    if dynamic_communities_df.empty:
        dynamic_communities_df = pd.DataFrame(columns=['country', 'year', 'community_id_dyn'])



Starting dynamic community detection for years 1988-2024...
  Processing communities for year: 1988
  Processing communities for year: 1989
  Processing communities for year: 1990
  Processing communities for year: 1991
  Processing communities for year: 1992
  Processing communities for year: 1993
  Processing communities for year: 1994
  Processing communities for year: 1995
  Processing communities for year: 1996
  Processing communities for year: 1997
  Processing communities for year: 1998
  Processing communities for year: 1999
  Processing communities for year: 2000
  Processing communities for year: 2001
  Processing communities for year: 2002
  Processing communities for year: 2003
  Processing communities for year: 2004
  Processing communities for year: 2005
  Processing communities for year: 2006
  Processing communities for year: 2007
  Processing communities for year: 2008
  Processing communities for year: 2009
  Processing communities for year: 2010
  Processing commun

## 5. Engineer Features from Dynamic Communities

Using the yearly community assignments, we create features that describe a country's community behavior over time. A key feature is **community stability**, which is calculated as `1 - rolling_mean(community_changed)` over a 3-year window. A value closer to 1 indicates a country has high stability (rarely changes trade blocs), while a value closer to 0 indicates frequent community shifts. This feature helps quantify the consistency of a country's trade alliances.

In [6]:
if not dynamic_communities_df.empty and 'community_id_dyn' in dynamic_communities_df.columns:
    print(f"\nEngineering features from dynamic communities (e.g., stability with window {ROLLING_WINDOW_DYN})...")
    dynamic_communities_df.sort_values(['country', 'year'], inplace=True)
    
    # Calculate 'community_changed' flag
    # Shift community_id to compare with the previous year's id for the same country
    dynamic_communities_df['prev_community_id_dyn'] = dynamic_communities_df.groupby('country')['community_id_dyn'].shift(1)
    # A change occurs if current_id is different from prev_id AND prev_id is not NaN (i.e., not the first year for the country)
    dynamic_communities_df['community_changed_dyn'] = ((dynamic_communities_df['community_id_dyn'] != dynamic_communities_df['prev_community_id_dyn']) & (dynamic_communities_df['prev_community_id_dyn'].notna())).astype(int)

    # Calculate community stability as 1 - mean of changes over the window
    dynamic_communities_df[f'community_stability_dyn'] = 1 - dynamic_communities_df.groupby('country')['community_changed_dyn'].transform(
        lambda x: x.rolling(window=ROLLING_WINDOW_DYN, min_periods=1).mean()
    )
    
    # Clean up intermediate columns
    dynamic_communities_df.drop(columns=['prev_community_id_dyn', 'community_changed_dyn'], inplace=True)
    
    print("\nCommunity-derived features (stability) added.")
    print("Sample of dynamic communities with stability features:")
    if sample_country_for_rolling_view and sample_country_for_rolling_view in dynamic_communities_df['country'].unique():
        print(dynamic_communities_df[dynamic_communities_df['country'] == sample_country_for_rolling_view].head(ROLLING_WINDOW_DYN + 2))
    else:
        print(dynamic_communities_df.head())
else:
    print("\nSkipping community feature engineering as `dynamic_communities_df` is empty or lacks 'community_id_dyn'.")


Engineering features from dynamic communities (e.g., stability with window 3)...

Community-derived features (stability) added.
Sample of dynamic communities with stability features:
         country  year  community_id_dyn  community_stability_dyn
467  Afghanistan  2000                -2                      1.0
546  Afghanistan  2001                -2                      1.0
624  Afghanistan  2002                -2                      1.0
775  Afghanistan  2003                -2                      1.0
845  Afghanistan  2004                -2                      1.0


## 6. Integrating Dynamic Features into Main Dataset

All the calculated dynamic features—centrality scores, their rolling statistics, community IDs, and community stability—are now merged back into the main DataFrame. This is done twice, once for the **importer** and once for the **exporter** in each trade pair, prefixing the new feature names accordingly. This step adds **34** new columns, expanding the DataFrame's shape to (17170, 825) and providing a rich, network-aware context for each trade observation.

In [7]:
print("\nStarting merge of dynamic features into the main DataFrame...")
original_df_cols = set(df.columns)

# --- Merge Dynamic Centralities ---
if not dynamic_centralities_df.empty and 'pagerank_dyn' in dynamic_centralities_df.columns:
    # Prepare for importer
    importer_centralities_to_merge = dynamic_centralities_df.copy()
    importer_cols_mapping = {col: f'importer_{col}' for col in importer_centralities_to_merge.columns if col not in ['country', 'year']}
    importer_centralities_to_merge.rename(columns=importer_cols_mapping, inplace=True)
    df = pd.merge(df, importer_centralities_to_merge, 
                  left_on=['importer', 'year'], right_on=['country', 'year'], 
                  how='left', suffixes=('', '_importer_drop'))
    df.drop(columns=[col for col in df.columns if '_importer_drop' in col or col == 'country'], errors='ignore', inplace=True)
    print("  Importer dynamic centrality features merged.")

    # Prepare for exporter
    exporter_centralities_to_merge = dynamic_centralities_df.copy()
    exporter_cols_mapping = {col: f'exporter_{col}' for col in exporter_centralities_to_merge.columns if col not in ['country', 'year']}
    exporter_centralities_to_merge.rename(columns=exporter_cols_mapping, inplace=True)
    df = pd.merge(df, exporter_centralities_to_merge, 
                  left_on=['exporter', 'year'], right_on=['country', 'year'], 
                  how='left', suffixes=('', '_exporter_drop'))
    df.drop(columns=[col for col in df.columns if '_exporter_drop' in col or col == 'country'], errors='ignore', inplace=True)
    print("  Exporter dynamic centrality features merged.")
else:
    print("  Skipping merge of dynamic centrality features as source DataFrame is empty or key columns missing.")

# --- Merge Dynamic Communities ---
if not dynamic_communities_df.empty and 'community_id_dyn' in dynamic_communities_df.columns:
    # Prepare for importer
    importer_communities_to_merge = dynamic_communities_df.copy()
    importer_comm_cols_mapping = {col: f'importer_{col}' for col in importer_communities_to_merge.columns if col not in ['country', 'year']}
    importer_communities_to_merge.rename(columns=importer_comm_cols_mapping, inplace=True)
    df = pd.merge(df, importer_communities_to_merge, 
                  left_on=['importer', 'year'], right_on=['country', 'year'], 
                  how='left', suffixes=('', '_importer_comm_drop'))
    df.drop(columns=[col for col in df.columns if '_importer_comm_drop' in col or col == 'country'], errors='ignore', inplace=True)
    print("  Importer dynamic community features merged.")

    # Prepare for exporter
    exporter_communities_to_merge = dynamic_communities_df.copy()
    exporter_comm_cols_mapping = {col: f'exporter_{col}' for col in exporter_communities_to_merge.columns if col not in ['country', 'year']}
    exporter_communities_to_merge.rename(columns=exporter_comm_cols_mapping, inplace=True)
    df = pd.merge(df, exporter_communities_to_merge, 
                  left_on=['exporter', 'year'], right_on=['country', 'year'], 
                  how='left', suffixes=('', '_exporter_comm_drop'))
    df.drop(columns=[col for col in df.columns if '_exporter_comm_drop' in col or col == 'country'], errors='ignore', inplace=True)
    print("  Exporter dynamic community features merged.")
else:
    print("  Skipping merge of dynamic community features as source DataFrame is empty or key columns missing.")

print("\nDynamic feature merging process complete.")
print(f"DataFrame shape after merging dynamic features: {df.shape}")
newly_added_cols = list(set(df.columns) - original_df_cols)
print(f"Number of newly added columns: {len(newly_added_cols)}")
print("Sample of newly added columns (first 5 if any):", newly_added_cols[:5])

print("\nDataFrame head after merging all dynamic features:")
print(df.head())
print("\nInfo for DataFrame after merges:")
df.info(max_cols=150, verbose=False, show_counts=True) # Show counts, but not full verbose to save space


Starting merge of dynamic features into the main DataFrame...
  Importer dynamic centrality features merged.
  Exporter dynamic centrality features merged.
  Importer dynamic community features merged.
  Exporter dynamic community features merged.

Dynamic feature merging process complete.
DataFrame shape after merging dynamic features: (17170, 825)
Number of newly added columns: 34
Sample of newly added columns (first 5 if any): ['importer_pagerank_dyn_roll_std', 'exporter_authority_score_dyn_roll_mean', 'exporter_hub_score_dyn_roll_std', 'importer_harmonic_centrality_dyn_roll_mean', 'exporter_authority_score_dyn']

DataFrame head after merging all dynamic features:
      importer        exporter  amount  year  importer_latitude  \
0  Afghanistan  American Samoa     0.0  2019               33.0   
1  Afghanistan         Andorra     0.0  2019               33.0   
2  Afghanistan       Argentina     0.0  2019               33.0   
3  Afghanistan       Australia     0.0  2019           

## 7. Lagging Dynamic Features for Predictive Modeling

A critical step before forecasting is to **lag** all newly created dynamic features. This ensures that to predict a value for year `Y`, we only use information available up to year `Y-1`, preventing data leakage. All **34** dynamic network features (e.g., `importer_pagerank_dyn`, `exporter_community_stability_dyn`) are shifted by one year for each `trade_pair_id`, creating a new set of `_lag1` features. The original, non-lagged versions are kept for potential analytical use, but only the lagged versions will be used for modeling.

In [8]:
# Identify all dynamic feature columns that were added (importer_ and exporter_ prefixed)
dynamic_cols_to_lag = [col for col in df.columns if (col.startswith('importer_') or col.startswith('exporter_')) and 
                       any(keyword in col for keyword in ['_dyn', '_roll_mean', '_roll_std', '_community_id_dyn', '_community_stability_dyn'])]
# Filter out columns that might already be lagged from previous parts (e.g. amount_lag_1)
dynamic_cols_to_lag = [col for col in dynamic_cols_to_lag if not col.endswith('_lag1') and not col.endswith('_lag2') and not col.endswith('_lag3')]

if dynamic_cols_to_lag:
    print(f"\nLagging {len(dynamic_cols_to_lag)} dynamic features by 1 year...")
    df.sort_values(['trade_pair_id', 'year'], inplace=True)
    
    for col in dynamic_cols_to_lag:
        df[f'{col}_lag1'] = df.groupby('trade_pair_id')[col].shift(1)
    
    print("\nDynamic features lagged.")
    lagged_col_names = [f'{col}_lag1' for col in dynamic_cols_to_lag]
    print("Sample of newly created lagged columns (first 5 if any):", lagged_col_names[:5])
    if lagged_col_names:
        print(df[['trade_pair_id', 'year'] + [c for c in lagged_col_names if c in df.columns][:3]].head(10))
    
    # Decision: Drop original non-lagged dynamic features to avoid confusion and using contemporaneous data in error.
    # This is generally safer for forecasting setup.
    # print(f"\nDropping {len(dynamic_cols_to_lag)} original non-lagged dynamic features.")
    # df.drop(columns=dynamic_cols_to_lag, inplace=True, errors='ignore')
    # For this assignment, we will keep them to show the process, but in Part 5, only lagged versions should be selected.
    print("Original non-lagged dynamic features are kept for now. Ensure only '*_lag1' versions are used for forecasting.")
else:
    print("\nNo dynamic features identified for lagging (this might be unexpected if previous steps ran correctly).")


Lagging 34 dynamic features by 1 year...

Dynamic features lagged.
Sample of newly created lagged columns (first 5 if any): ['importer_pagerank_dyn_lag1', 'importer_hub_score_dyn_lag1', 'importer_authority_score_dyn_lag1', 'importer_harmonic_centrality_dyn_lag1', 'importer_betweenness_centrality_dyn_lag1']
                trade_pair_id  year  importer_pagerank_dyn_lag1  \
0  Afghanistan_American Samoa  2019                         NaN   
1         Afghanistan_Andorra  2019                         NaN   
2       Afghanistan_Argentina  2019                         NaN   
3       Afghanistan_Australia  2019                         NaN   
4         Afghanistan_Austria  2019                         NaN   
5      Afghanistan_Azerbaijan  2019                         NaN   
6         Afghanistan_Bahrain  2019                         NaN   
7      Afghanistan_Bangladesh  2019                         NaN   
8         Afghanistan_Belarus  2019                         NaN   
9         Afghanistan

## 8. Handle Missing Values from New Lagged/Rolled Features

The lagging process inevitably introduces `NaN` values for the first observation of each time series. To ensure the dataset is complete, we apply a simple and fast **median imputation** strategy to fill these missing values in the **34** newly created lagged dynamic feature columns. This pragmatic approach prepares the data for modeling without the computational overhead of re-running a full MICE imputation.

In [9]:
# Identify all columns that are either rolling stats of dynamic features or lagged dynamic features
cols_for_new_imputation = [col for col in df.columns if ('_roll_mean' in col or '_roll_std' in col or '_lag1' in col) and 
                             any(keyword in col for keyword in ['_dyn', '_community_id_dyn', '_community_stability_dyn'])]
cols_for_new_imputation = [col for col in cols_for_new_imputation if df[col].isnull().any()] # Only impute if NaNs exist

if cols_for_new_imputation:
    print(f"\nImputing NaNs in {len(cols_for_new_imputation)} newly created/lagged dynamic feature columns using median imputation.")
    nan_counts_before = df[cols_for_new_imputation].isnull().sum()
    
    for col in cols_for_new_imputation:
        median_val = df[col].median()
        if pd.isna(median_val):
             # If median is NaN (e.g., all values are NaN), fill with 0 or another default
            df[col].fillna(0, inplace=True) 
            print(f"    NaNs in {col} (all NaN) imputed with 0. Original NaN count: {nan_counts_before[col]}")
        else:
            df[col].fillna(median_val, inplace=True)
            # print(f"    NaNs in {col} imputed with median {median_val:.4f}. Original NaN count: {nan_counts_before[col]}")
            
    print("\nNaN imputation for new dynamic features complete.")
    nan_counts_after = df[cols_for_new_imputation].isnull().sum().sum()
    print(f"Total NaNs remaining in these specific imputed columns: {nan_counts_after}")
    if nan_counts_after > 0:
        print("Warning: Some NaNs may remain if entire columns were NaN.")
else:
    print("\nNo NaNs found in newly created/lagged dynamic feature columns, or no such columns identified for imputation.")


Imputing NaNs in 34 newly created/lagged dynamic feature columns using median imputation.

NaN imputation for new dynamic features complete.
Total NaNs remaining in these specific imputed columns: 0


C:\Users\Askeladd\AppData\Local\Temp\ipykernel_30800\1088823633.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(median_val, inplace=True)
C:\Users\Askeladd\AppData\Local\Temp\ipykernel_30800\1088823633.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

## 9. Final Review and Save Processed Data

The DataFrame is now significantly enriched with features capturing the dynamic evolution of the trade network. After a final review confirming the new shape of **(17170, 859)** and that all missing values have been handled, the DataFrame is saved to a new CSV file, `trade_data_dynamic_features.csv`. This file will serve as the primary input for Part 5: Predictive Modeling.

In [10]:
print("\nFinal review of the DataFrame before saving...")
print(f"Final DataFrame shape: {df.shape}")
print("Number of unique trade pairs:", df['trade_pair_id'].nunique())
print("Year range in final data:", df['year'].min(), "to", df['year'].max())

print("\nChecking for any remaining NaNs in the entire DataFrame (should be minimal if Part 2 MICE was effective and new imputation handled its scope):")
total_nans_final = df.isnull().sum().sum()
print(f"Total NaNs in the final DataFrame: {total_nans_final}")
if total_nans_final > 0:
    print("Columns with remaining NaNs:")
    print(df.isnull().sum()[df.isnull().sum() > 0])

df.to_csv(FINAL_OUTPUT_FILE_PART4, index=False)
print(f"\nFinal feature-engineered data with dynamic network features saved to: {FINAL_OUTPUT_FILE_PART4}")

print("\nFinal DataFrame Info (summary):")
df.info(verbose=False, show_counts=True)


Final review of the DataFrame before saving...
Final DataFrame shape: (17170, 859)
Number of unique trade pairs: 3681
Year range in final data: 1988 to 2024

Checking for any remaining NaNs in the entire DataFrame (should be minimal if Part 2 MICE was effective and new imputation handled its scope):
Total NaNs in the final DataFrame: 0

Final feature-engineered data with dynamic network features saved to: trade_data_dynamic_features.csv

Final DataFrame Info (summary):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17170 entries, 0 to 17169
Columns: 859 entries, importer to exporter_community_stability_dyn_lag1
dtypes: float64(853), int32(1), int64(2), object(3)
memory usage: 112.5+ MB


## End of Part 4

This notebook has successfully executed a comprehensive dynamic network analysis. We have:
- Constructed yearly trade networks and calculated dynamic centrality measures (PageRank, HITS, etc.) for each country.
- Engineered rolling statistics for these centralities to capture trends and stability.
- Performed dynamic community detection using the Louvain algorithm and derived features like community stability.
- Merged these rich, dynamic, network-based features back into our main dataset for both importer and exporter roles.
- Lagged all newly created dynamic features by one year to ensure their suitability for forecasting.
- Addressed all new missing values using median imputation.

The resulting DataFrame, saved as `trade_data_dynamic_features.csv`, is now substantially enhanced with features that reflect the evolving structure and interconnectedness of the global trade system. This dataset forms a strong foundation for Part 5, where we will focus on building and evaluating predictive models.